In [ ]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, select
import datetime

# Database connection setup
con = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=HF-BD-SPV1\MSSQL;'
    r'DATABASE=MASTR_1;'
    r'Trusted_Connection=yes;'
    r'Encrypt=optional'
)

# Create SQLAlchemy engine and metadata
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={con}')
metadata = MetaData()

# Define table and query
query = Table('EinheitenWind', metadata, autoload_with=engine)

# Query the table to retrieve necessary columns, with filter for Lage == 888
stmt = select(
    query.c.Inbetriebnahmedatum,
    query.c.DatumEndgueltigeStilllegung,
    query.c.Bruttoleistung,
    query.c.EinheitBetriebsstatus
).where(query.c.Lage == 888)

# Execute query and load into DataFrame
df = pd.read_sql(stmt, engine)

# Create a complete DataFrame of months since January 2020
months = pd.date_range('2020-01-01', datetime.datetime.today(), freq='MS').to_period('M')
months_df = pd.DataFrame(months, columns=['Month'])

# Create two new columns 'Month_Installed' and 'Month_Decommissioned' with the year-month format for installation and decommissioning
df['Month_Installed'] = df['Inbetriebnahmedatum'].dt.to_period('M')
df['Month_Decommissioned'] = df['DatumEndgueltigeStilllegung'].dt.to_period('M')
df = df.drop(['Inbetriebnahmedatum','DatumEndgueltigeStilllegung'],axis=1)

# Filter for installations and decommissionings separately
installations_df = df[df['EinheitBetriebsstatus'] == 35]
decommissionings_df = df[df['EinheitBetriebsstatus'] == 38]

# Group installations and decommissionings by their respective months and sum the 'Bruttoleistung'
installations_summed = installations_df.groupby('Month_Installed')['Bruttoleistung'].sum().reset_index(name='Bruttoleistung_Installed')
decommissionings_summed = decommissionings_df.groupby('Month_Decommissioned')['Bruttoleistung'].sum().reset_index(name='Bruttoleistung_Decommissioned')

# Merge the summed installations and decommissionings with the complete month range DataFrame
result_df = months_df.merge(installations_summed, left_on='Month', right_on='Month_Installed', how='left').merge(decommissionings_summed, left_on='Month', right_on='Month_Decommissioned', how='left')

# Drop the 'Month_Installed' and 'Month_Decommissioned' columns, and fill NaN values with 0
result_df.drop(columns=['Month_Installed', 'Month_Decommissioned'], inplace=True)
result_df.fillna(0, inplace=True)
result_df['Nettozubau'] = result_df['Bruttoleistung_Installed'] - result_df['Bruttoleistung_Decommissioned']

print(result_df)

path = r"C:\Users\Christopher.Troost\BDEW Verband der Energie- und Wasse\BDEW Volkswirtschaft - Data Lab\Dev\CT\BDEW-Showroom"

# Save the DataFrame to a CSV file
result_df.to_csv(path + '\Wind_Onshore_Development_monthly.csv', float_format='%.2f', decimal=',', index=False)

#----------------------

import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, select
import datetime

# Database connection setup
con = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=HF-BD-SPV1\MSSQL;'
    r'DATABASE=MASTR_1;'
    r'Trusted_Connection=yes;'
    r'Encrypt=optional'
)

# Create SQLAlchemy engine and metadata
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={con}')
metadata = MetaData()

# Define table and query
query = Table('EinheitenWind', metadata, autoload_with=engine)

# Query the table to retrieve necessary columns, with filter for Lage == 888
stmt = select(
    query.c.Inbetriebnahmedatum,
    query.c.DatumEndgueltigeStilllegung,
    query.c.Bruttoleistung,
    query.c.EinheitBetriebsstatus
).where(query.c.Lage == 888)

# Execute query and load into DataFrame
df = pd.read_sql(stmt, engine)

# Create a new column 'Year_Installed' and 'Year_Decommissioned' with just the year for installation and decommissioning
df['Year_Installed'] = df['Inbetriebnahmedatum'].dt.year
df['Year_Decommissioned'] = df['DatumEndgueltigeStilllegung'].dt.year
df = df.drop(['Inbetriebnahmedatum','DatumEndgueltigeStilllegung'], axis=1)

# Filter for installations and decommissionings separately
installations_df = df[df['EinheitBetriebsstatus'] == 35]
decommissionings_df = df[df['EinheitBetriebsstatus'] == 38]

# Group installations and decommissionings by year and sum the 'Bruttoleistung'
installations_summed = installations_df.groupby('Year_Installed')['Bruttoleistung'].sum().reset_index(name='Bruttoleistung_Installed')
decommissionings_summed = decommissionings_df.groupby('Year_Decommissioned')['Bruttoleistung'].sum().reset_index(name='Bruttoleistung_Decommissioned')

# Create a complete DataFrame of years
years = pd.DataFrame({'Year': range(2020, datetime.datetime.today().year + 1)})

# Merge the summed installations and decommissionings with the complete year range DataFrame
result_df = years.merge(installations_summed, left_on='Year', right_on='Year_Installed', how='left').merge(decommissionings_summed, left_on='Year', right_on='Year_Decommissioned', how='left')

# Drop the 'Year_Installed' and 'Year_Decommissioned' columns, and fill NaN values with 0
result_df.drop(columns=['Year_Installed', 'Year_Decommissioned'], inplace=True)
result_df.fillna(0, inplace=True)

# Calculate net capacity additions (Nettozubau)
result_df['Nettozubau'] = result_df['Bruttoleistung_Installed'] - result_df['Bruttoleistung_Decommissioned']

# Print the result
print(result_df)

# Save the DataFrame to a CSV file
path = r"C:\Users\Christopher.Troost\BDEW Verband der Energie- und Wasse\BDEW Volkswirtschaft - Data Lab\Dev\CT\BDEW-Showroom"
result_df.to_csv(path + '\Wind_Onshore_Development_annual.csv', float_format='%.2f', decimal=',', index=False)


In [ ]:
df